# Setting up CLI tool

## Do before running this Notebook:
- Obtain WMLA CLI `dlim` CLI tool (patch from dev team)
- Suggestion: place in `$HOME/bin`
- Make executable `chmod a+x $HOME/bin/dlim`

## Do in this Notebook
- Place directory on `$PATH`: `os.environ['PATH'] = os.environ['PATH'] + f':{os.environ["HOME"]}/bin'`

## Sample usage
```!dlim model list --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN```

# 1. Setup

In [ ]:
import os
import json
import requests

In [ ]:
# Set deployment name
deploy_name = 'pingpong1'

In [ ]:
# REST_SERVER required by dlim CLI tool
os.environ['HOST'] = 'https://wmla-console-cpd-wmla.apps.cpd.mskcc.org'
os.environ['REST_SERVER'] = os.environ['HOST'] + '/dlim/v1/'
os.environ['INFERENCE_URL'] = 'https://wmla-inference-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/inference'
    
# Add to path
if not f"{os.environ['HOME']}/bin" in os.environ['HOME']:
    os.environ['PATH'] = os.environ['PATH'] + f':{os.environ["HOME"]}/bin'

# 2. Set up WMLA deployment dependencies

## 2.1 Create working directory

In [ ]:
os.environ['DEPLOY_NAME'] = deploy_name
os.environ['DIR_DEPLOY_SUBMISSION'] = f'/userfs/deploy_submissions/{os.environ["DEPLOY_NAME"]}'

!rm -rf $DIR_DEPLOY_SUBMISSION
!mkdir -p $DIR_DEPLOY_SUBMISSION

## 2.2 Create model JSON file

In [ ]:
model_profile = {
    "name" : os.environ['DEPLOY_NAME'],
    "tag" : "test",
    "weight_path" : "N/A",
    "runtime" : "python",
    "kernel_path" : "kernel.py",
    "schema_version" : "1"
}

# Save to working directory
with open(f'{os.environ["DIR_DEPLOY_SUBMISSION"]}/model.json','w') as f:
    json.dump(model_profile, f)

## 2.3 Create README file

In [ ]:
with open(f'{os.environ["DIR_DEPLOY_SUBMISSION"]}/README.md','w') as f:
    f.write('Test.')

## 2.4 Create kernel file

In [ ]:
kernel_file = """#!/usr/bin/env python

import redhareapiversion
from redhareapi import Kernel
import json

class TestKernel(Kernel):

    def on_task_invoke(self, task_context):
        try:
            Kernel.log_debug("on_task_invoke")
            # using loop to handle batch-size
            while task_context != None:
                Kernel.log_debug("task id: %s " % task_context.get_id())
                input_data = json.loads(task_context.get_input_data())
                if 'seq' in input_data:
                    Kernel.log_debug("sequence number: %s " % input_data['seq'])
                task_context.set_output_data(json.dumps(input_data))
                task_context = task_context.next()
        except Exception as e:
            Kernel.log_error(str(e))

if __name__ == '__main__':
    ppkernel = TestKernel()
    ppkernel.run()"""

print(kernel_file, file=open(f'{os.environ["DIR_DEPLOY_SUBMISSION"]}/kernel.py', 'w'))

## 2.5 Submit deployment

In [ ]:
# Remove hidden files
!rm -rf $DIR_DEPLOY_SUBMISSION/.*

In [ ]:
!dlim model deploy -p $DIR_DEPLOY_SUBMISSION --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

In [ ]:
## ensure we see deployment
!dlim model list --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

In [ ]:
!dlim model view $DEPLOY_NAME --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

In [ ]:
!dlim model viewprofile $DEPLOY_NAME --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

## 2.6 Start deployment

In [ ]:
!dlim model start $DEPLOY_NAME --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

In [ ]:
!dlim model view $DEPLOY_NAME -s --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

## 2.6 Obtain REST URI

In [ ]:
!dlim model view $DEPLOY_NAME -a --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

# 3. Score against deployment

In [ ]:
headers = {'Authorization': f'Bearer {os.getenv("USER_ACCESS_TOKEN")}'}
data = {'data':'123'}
url = f'{os.environ["INFERENCE_URL"]}/{os.environ["DEPLOY_NAME"]}'

In [ ]:
r = requests.post(url, headers=headers,
                  json = data, verify = False)
r.status_code

In [ ]:
r.json()

# 4. Clean up / Miscellaneous

- **NOTE**: These commands require user input (i.e., confirmation) so cannot run from JupyterLab